https://github.com/microsoft/recommenders/blob/main/examples/01_prepare_data/data_split.ipynb

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
transaction = pd.read_csv('book_transactions.csv', encoding='cp949')

In [3]:
data = transaction[['회원번호', '책제목']]
ids = data.groupby('회원번호').책제목.count().reset_index().query("책제목 > 20").회원번호.unique()
data = data.query('회원번호 in @ids')

id2user = dict(enumerate(data.회원번호.unique()))
user2id = {j:i for i, j in id2user.items()}
id2book = dict(enumerate(data.책제목.unique()))
book2id = {j:i for i, j in id2book.items()}
#id2cat = dict(enumerate(data.카테고리.unique()))
#cat2id = {j:i for i, j in id2cat.items()}

data.회원번호 = data.회원번호.map(lambda x: user2id[x])
data.책제목 = data.책제목.map(lambda x: book2id[x])
data = data.drop_duplicates()

In [4]:
#data_cat = transaction.groupby('책제목')['카테고리'].unique().map(lambda x: x[0]).reset_index()

In [5]:
#data_cat['책제목'] = data_cat.책제목.map(lambda x: book2id[x])
#data_cat['카테고리'] = data_cat.카테고리.map(lambda x: cat2id[x])

In [6]:
#data_cat = dict(data_cat.values)

In [7]:
data.to_csv('main_data.csv', index=False)

In [8]:
#data_cat = transaction[['회원번호', '카테고리']]
#data_cat.회원번호 = data_cat.회원번호.map(lambda x: user2id[x])
#data_cat.카테고리 = data_cat.카테고리.map(lambda x: cat2id[x])
#data_cat = data_cat.drop_duplicates()

In [9]:
data_tr = pd.read_csv('main_data.csv')

In [10]:
data_tr.shape

(109553, 2)

In [11]:
train_data = data_tr.copy()

In [12]:
items = data.groupby('회원번호')['책제목'].unique()
test_data = items.map(lambda x: x[-1]).reset_index().values
train_data = train_data.append(pd.DataFrame(test_data, columns=['회원번호', '책제목'])).drop_duplicates(keep=False)

In [13]:
train_data.to_csv('main_data_train.csv', index=False)

In [14]:
item_num = data_tr.책제목.nunique()
item_list = set(range(item_num))
def noninteracted(x):
    lst = list(item_list - set(x))
    sampled = np.random.choice(lst, 99)
    result = x[-1].reshape(1).tolist() + sampled.tolist()
    return result
res = items.map(noninteracted)

In [15]:
np.save('main_data_test.npy', res.reset_index().values, allow_pickle=True)

In [16]:
test_data_raw = np.load('main_data_test.npy', allow_pickle=True)

In [17]:
test_data = []
for data in test_data_raw:
    u = data[0]
    i = data[1]
    for item in i:
        test_data.append([u, int(item)])

In [21]:
print(data_tr.회원번호.nunique(), data_tr.책제목.nunique())

3068 42929
